## BLERSSI Pipeline Deployment in UCS

### Clone git repo

In [1]:
BRANCH_NAME="dev" #Provide git branch "master" or "dev"
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 4718 (delta 42), reused 125 (delta 30), pack-reused 4567
Receiving objects: 100% (4718/4718), 18.41 MiB | 42.37 MiB/s, done.
Resolving deltas: 100% (1771/1771), done.


In [2]:
!pip install kfp --user

     |████████████████████████████████| 116 kB 17.9 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 6.5 MB/s  eta 0:00:01
     |████████████████████████████████| 51 kB 3.4 MB/s  eta 0:00:01
  Created wheel for kfp: filename=kfp-1.0.0-py3-none-any.whl size=160799 sha256=44709e1c73af752dea404cea1761e8bd497aa8456cbed54b78504c9d68382279
  Stored in directory: /home/jovyan/.cache/pip/wheels/a1/4d/1d/532b25d0ea5bf28d352c8973b8ce9b1d7f8f5220b236af1c62
  Created wheel for kfp-server-api: filename=kfp_server_api-1.0.0-py3-none-any.whl size=105028 sha256=9cd6ff7ac1bce0ce21850d40b29ff55d9f64df696f3831375c5226990fd91233
  Stored in directory: /home/jovyan/.cache/pip/wheels/c9/65/08/c985cd0191974934cd16f91b4c43a83779b90e87c0773ada32
  Created wheel for strip-hints: filename=strip_hints-0.1.9-py2.py3-none-any.whl size=24671 sha256=165f32105991de097523e3110ee2f284cddba6c7a61509ae7f42ea74fa144486
  Stored in directory: /home/jovyan/.cache/pip/wheels/21/6d/fa/7ed7c0560e1ef39ebabd5cc0241e

## Restart Notebook Kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

### Import libraries

In [1]:
import kfp
import os
import yaml
import calendar
import time
import pandas as pd
import numpy as np
import requests
import json

from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from kubernetes.client.rest import ApiException

k8s_config.load_incluster_config()
api_client = k8s_client.CoreV1Api()
custom_api=k8s_client.CustomObjectsApi()

### Component files Declarations

In [2]:
path='cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem/pipelines/'
component_root_katib= path+'components/v2/tf-katib/'
component_root_train= path+'components/v2/seldon/'

#### Components Description

tf_katib_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - &nbsp;&nbsp;&nbsp; Run katib for BLERSSI to get optimal hyperparamater values and use it to train model. <br>
tf_train_model_op &nbsp;&nbsp;&nbsp; - &nbsp; Load dataset from nfs-volume, train BLERSSI model  and save model in nfs-volume. <br> 

To [build](https://github.com/CiscoAI/cisco-kubeflow-starter-pack/tree/dev/apps/networking/ble-localization/onprem/seldon/model-server) the docker image and push into your Docker Hub. It will be used when adding a new inference server

## Adding a new inference server 
The list of available inference servers in Seldon Core is maintained in the **seldon-config** configmap, which lives in the same namespace as your Seldon Core operator. In particular, the **predictor_servers** key holds the JSON config for each inference server. 

[Refer to for more information](https://docs.seldon.io/projects/seldon-core/en/v1.1.0/servers/custom.html)

In [3]:
api_client.patch_namespaced_config_map(name="seldon-config", namespace="kubeflow",pretty=True, body={"data":{"predictor_servers":'{"MLFLOW_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/mlflowserver_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/mlflowserver_rest"}},"SKLEARN_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/sklearnserver_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/sklearnserver_rest"}},"TENSORFLOW_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/tfserving-proxy_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/tfserving-proxy_rest"},"tensorflow":true,"tfImage":"tensorflow/serving:2.1.0"},"XGBOOST_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/xgboostserver_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/xgboostserver_rest"}}, "CUSTOM_INFERENCE_SERVER":{"rest":{"defaultImageVersion":"1.0","image":"samba07/blerssi-seldon"}}}'}})

{'api_version': 'v1',
 'binary_data': None,
 'data': {'credentials': '{"gcs":{"gcsCredentialFileName":"gcloud-application-credentials.json"},"s3":{"s3AccessKeyIDName":"awsAccessKeyID","s3SecretAccessKeyName":"awsSecretAccessKey"}}',
          'explainer': '{"image":"seldonio/alibiexplainer:1.2.1"}',
          'predictor_servers': '{"MLFLOW_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/mlflowserver_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/mlflowserver_rest"}},"SKLEARN_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/sklearnserver_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/sklearnserver_rest"}},"TENSORFLOW_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/tfserving-proxy_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/tfserving-proxy_rest"},"tensorflow":true,"tfImage":"tensorflow/serving:2.1.0"},"XGBOOST_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/xgboostserver_

### Load components from respective .YAML config files

In [4]:
tf_katib_op = kfp.components.load_component_from_file(os.path.join(component_root_katib, 'component.yaml')) 
tf_train_model_op = kfp.components.load_component_from_file(os.path.join(component_root_train, "component.yaml"))

## Define SeldonDeployment
Create an SeldonDeployment with a blerssi model

In [5]:
MODEL_PATH="Model_Blerssi"
# Creating timestamp
timestamp = str(calendar.timegm(time.gmtime()))
print(timestamp)
seldon=f"""apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: blerssi-{timestamp}
spec:
  name: blerssi
  predictors:
  - graph:
      children: []
      implementation: CUSTOM_INFERENCE_SERVER
      modelUri: pvc://nfs/{MODEL_PATH}
      name: blerssi
    explainer:
      containerSpec:
          image: seldonio/alibiexplainer:1.2.2-dev
          name: explainer
      type: AnchorTabular
      modelUri: pvc://nfs/{MODEL_PATH}
    name: default
    replicas: 1
"""
seldon_resource=yaml.safe_load(seldon)

1596093702


### Define Volume and Volume Mounts

In [6]:
nfs_pvc = k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = k8s_client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = k8s_client.V1VolumeMount(mount_path='/mnt/', name='nfs')

### Define pipeline function

In [7]:
def blerssi_pipeline():
    #Defining Task to run katib
    tf_katib_task = tf_katib_op(timestamp=timestamp, image="docker.io/samba07/blerssi-seldon-train:0.1")
    tf_katib_task.add_volume(nfs_volume)
    tf_katib_task.add_volume_mount(nfs_volume_mount)    
    
    #Defining Task for Model training
    model_dir=os.path.join("/mnt",MODEL_PATH)
    export_dir=os.path.join("/mnt",MODEL_PATH)
    tf_train_model_task = tf_train_model_op(timestamp=timestamp, tf_model_dir=model_dir, tf_export_dir=export_dir)
    tf_train_model_task.add_volume(nfs_volume)
    tf_train_model_task.add_volume_mount(nfs_volume_mount)
    tf_train_model_task.after(tf_katib_task)

    seldon_serv=kfp.dsl.ResourceOp(name="seldon deploy",k8s_resource=seldon_resource)
    seldon_serv.add_volume(nfs_volume)
    seldon_serv.after(tf_train_model_task)
    
#Creating a pipeline run
kfp.Client().create_run_from_pipeline_func(blerssi_pipeline, arguments={})

RunPipelineResult(run_id=3b4eb7a7-e1c1-4765-afb2-234d20eb2026)

## Run a Prediction
Before run a prediction, make sure that Pipeline Run is Complete in the Dashboard

In [8]:
#First we can check if our Seldon deployment is running
!kubectl -n kubeflow get seldondeployment

NAME                 AGE
blerssi-1596093702   6m39s


In [9]:
%%bash --out SELDON_DEP_NAME
echo "$(kubectl -n kubeflow get seldondeployment -o jsonpath='{.items[0].metadata.name}')"

### Wait for state to become available

In [10]:
status=False
while True:
    seldon_status=custom_api.get_namespaced_custom_object_status(group="machinelearning.seldon.io", version="v1alpha2", namespace="kubeflow", plural="seldondeployments", name=SELDON_DEP_NAME.strip())
    if seldon_status["status"]["state"] == "Available":
        status=True
        print("Status: %s"%seldon_status["status"]["state"])
    if status:
        break
    print("Status: %s"%seldon_status["status"]["state"])
    time.sleep(30)

Status: Available


In [11]:
CLUSTER='ucs' #where your cluster running 'gcp' or 'ucs'

In [12]:
%%bash -s "$CLUSTER" --out NODE_IP
if [ $1 = "ucs" ]
then
    echo "$(kubectl get node -o=jsonpath='{.items[0].status.addresses[0].address}')"
else
    echo "$(kubectl get node -o=jsonpath='{.items[0].status.addresses[1].address}')"
fi

In [13]:
%%bash --out INGRESS_PORT
INGRESS_GATEWAY="istio-ingressgateway"
echo "$(kubectl -n istio-system get service $INGRESS_GATEWAY -o jsonpath='{.spec.ports[1].nodePort}')"

### Data for prediction

In [14]:
path="cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem"
df_full = pd.read_csv(os.path.join(path,'data/iBeacon_RSSI_Unlabeled_truncated.csv')) #Labeled dataset
COLUMNS = list(df_full.columns)
FEATURES = COLUMNS[2:]
  # Input Data Preprocessing 
df_full = df_full.drop(['date'],axis = 1)
df_full = df_full.drop(['location'],axis = 1)
df_full[FEATURES] = (df_full[FEATURES])/(-200)
input_data=df_full.to_numpy()[:1]
input_data

array([[1.   , 1.   , 0.4  , 1.   , 0.385, 0.28 , 0.405, 1.   , 1.   ,
        1.   , 1.   , 1.   , 1.   ]])

In [15]:
headers={"Content-Type": "application/json"}
def inference_predict(X):
    data={"data":{"ndarray":X.tolist()}}
    url = f"http://{NODE_IP.strip()}:{INGRESS_PORT.strip()}/seldon/kubeflow/{SELDON_DEP_NAME.strip()}/api/v1.0/predictions"
    response=requests.post(url, data=json.dumps(data), headers=headers)
    probabilities=response.json()['data']['ndarray']
    for prob in probabilities:
        cls_id=np.argmax(prob)
        print("Probability: %s"%prob[cls_id])
        print("Class-id: %s"%cls_id)

def explain(X):
    if np.ndim(X)==2:
        data={"data":{"ndarray":X.tolist()}}
    else:
        data={"data":{"ndarray":[X.tolist()]}}
    url = f"http://{NODE_IP.strip()}:{INGRESS_PORT.strip()}/seldon/kubeflow/%s-explainer/default/api/v1.0/explain"%SELDON_DEP_NAME.strip()
    response=requests.post(url, data=json.dumps(data), headers=headers)
    print('Anchor: %s' % (' AND '.join(response.json()['names'])))
    print('Coverage: %.2f' % response.json()['coverage'])

In [16]:
inference_predict(input_data)

Probability: 0.07242913544178009
Class-id: 52


## Prediction of the model and explain

In [17]:
explain(input_data)

Anchor: 0.39 < b3002 <= 1.00 AND 0.40 < b3004 <= 1.00 AND b3009 <= 1.00 AND b3012 <= 1.00 AND b3011 <= 1.00 AND b3013 <= 1.00 AND b3006 <= 1.00 AND b3003 <= 1.00 AND b3010 <= 1.00 AND b3005 <= 1.00 AND b3001 <= 1.00 AND b3007 <= 1.00 AND b3008 <= 1.00
Coverage: 0.48


## CleanUp
### Delete Seldon Serving Deployment

In [18]:
custom_api.delete_namespaced_custom_object(group="machinelearning.seldon.io", version="v1alpha2", namespace="kubeflow", plural="seldondeployments", name=SELDON_DEP_NAME.strip(), body=k8s_client.V1DeleteOptions())

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'blerssi-1596093702',
  'group': 'machinelearning.seldon.io',
  'kind': 'seldondeployments',
  'uid': '0cff0dc8-93a0-49e8-99af-ddaaa7b91be4'}}